R1CS to QAP to BulletProof
=============

## Example

In [1]:
# ref: https://medium.com/@VitalikButerin/quadratic-arithmetic-programs-from-zero-to-hero-f6d558cea649

from functools import partial
from klefki.zkp.r1cs import R1CS, mul
from klefki.zkp.qap import R1CS2QAP, proof, verify
from klefki.curves.secp256k1 import FiniteFieldCyclicSecp256k1 as F
import ast



In [2]:
# map int to field
ciphers = [1,2,3,4,5,6,7,8,9]
times = 5

@R1CS.r1cs(F)
def f(x, k, c):
    y = x + c + k
    return x ** 3

@R1CS.r1cs(F, globals())
def mimc(x, k):
    for i in range(times):
        c = ciphers[i]
        x = f(x, k, c)
    return x + k

Exception: Using a variable before it is set!

In [3]:
f.flatcode

NameError: name 'f' is not defined

In [4]:
mimc.flatcode

[['set', 'c', 1],
 ['+', 'Local<Rc(0)>Sym::1', 'Local<Rc(0)>x::0', 2],
 ['+', 'Local<Rc(0)>x::0', 'Local<Rc(0)>Sym::1', 1],
 ['*', 'Local<Rc(0)>Sym::2', 'Local<Rc(0)>x::0', 'Local<Rc(0)>x::0'],
 ['*', 'x::0', 'Local<Rc(0)>Sym::2', 'Local<Rc(0)>x::0'],
 ['set', 'c::1', 2],
 ['+', 'Local<Rc(1)>Sym::1', 'Local<Rc(1)>x::0', 2],
 ['+', 'Local<Rc(1)>x::0', 'Local<Rc(1)>Sym::1', 1],
 ['*', 'Local<Rc(1)>Sym::2', 'Local<Rc(1)>x::0', 'Local<Rc(1)>x::0'],
 ['*', 'x::2', 'Local<Rc(1)>Sym::2', 'Local<Rc(1)>x::0'],
 ['set', 'c::3', 3],
 ['+', 'Local<Rc(2)>Sym::1', 'Local<Rc(2)>x::0', 2],
 ['+', 'Local<Rc(2)>x::0', 'Local<Rc(2)>Sym::1', 1],
 ['*', 'Local<Rc(2)>Sym::2', 'Local<Rc(2)>x::0', 'Local<Rc(2)>x::0'],
 ['*', 'x::4', 'Local<Rc(2)>Sym::2', 'Local<Rc(2)>x::0'],
 ['set', 'c::5', 4],
 ['+', 'Local<Rc(3)>Sym::1', 'Local<Rc(3)>x::0', 2],
 ['+', 'Local<Rc(3)>x::0', 'Local<Rc(3)>Sym::1', 1],
 ['*', 'Local<Rc(3)>Sym::2', 'Local<Rc(3)>x::0', 'Local<Rc(3)>x::0'],
 ['*', 'x::6', 'Local<Rc(3)>Sym::2', 'Loc

The format of a flatcode line is:

$$
\left[Op, Out, S_a, S_b\right]
$$

In [5]:
mimc.var

['~one',
 'x',
 'k',
 '~out',
 'c',
 'Local<Rc(0)>x::0',
 'Local<Rc(0)>x::1',
 'Local<Rc(0)>Sym::2',
 'x::0',
 'c',
 'Local<Rc(1)>x::0',
 'Local<Rc(1)>x::1',
 'Local<Rc(1)>Sym::2',
 'x::1',
 'c',
 'Local<Rc(2)>x::0',
 'Local<Rc(2)>x::1',
 'Local<Rc(2)>Sym::2',
 'x::2',
 'c',
 'Local<Rc(3)>x::0',
 'Local<Rc(3)>x::1',
 'Local<Rc(3)>Sym::2',
 'x::3',
 'c',
 'Local<Rc(4)>x::0',
 'Local<Rc(4)>x::1',
 'Local<Rc(4)>Sym::2',
 'x::4']

The format of variable is

$$
[One, Input_0, \cdots, Input_n, Output, S_0, S_1, \cdots, S_n]
$$


In [6]:
assert len(mimc.A[0]) == len(mimc.var)

For each line of flatcodes, we have $A_i.s \circ B_i.s == C_i.s$

In [7]:
s = mimc.witness(F(2))
s

[FiniteFieldCyclicSecp256k1::1,
 FiniteFieldCyclicSecp256k1::2,
 FiniteFieldCyclicSecp256k1::0,
 FiniteFieldCyclicSecp256k1::27,
 FiniteFieldCyclicSecp256k1::5,
 FiniteFieldCyclicSecp256k1::2,
 FiniteFieldCyclicSecp256k1::3,
 FiniteFieldCyclicSecp256k1::9,
 FiniteFieldCyclicSecp256k1::27,
 FiniteFieldCyclicSecp256k1::0,
 FiniteFieldCyclicSecp256k1::2,
 FiniteFieldCyclicSecp256k1::3,
 FiniteFieldCyclicSecp256k1::9,
 FiniteFieldCyclicSecp256k1::27,
 FiniteFieldCyclicSecp256k1::0,
 FiniteFieldCyclicSecp256k1::2,
 FiniteFieldCyclicSecp256k1::3,
 FiniteFieldCyclicSecp256k1::9,
 FiniteFieldCyclicSecp256k1::27,
 FiniteFieldCyclicSecp256k1::0,
 FiniteFieldCyclicSecp256k1::2,
 FiniteFieldCyclicSecp256k1::3,
 FiniteFieldCyclicSecp256k1::9,
 FiniteFieldCyclicSecp256k1::27,
 FiniteFieldCyclicSecp256k1::0,
 FiniteFieldCyclicSecp256k1::2,
 FiniteFieldCyclicSecp256k1::3,
 FiniteFieldCyclicSecp256k1::9,
 FiniteFieldCyclicSecp256k1::27]

In [8]:
sum(mul(mimc.A[0], s)) * sum(mul(mimc.B[0], s)) == sum(mul(mimc.C[0], s))

False

## Gen R1CS over $F_q$

In [9]:
r1cs = mimc.r1cs
r1cs

([[FiniteFieldCyclicSecp256k1::115792089237316195423570985008687907852837564279074904382605163141518161494336,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::1,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   FiniteFieldCyclicSecp256k1::0,
   Fi

In [10]:
QAP = R1CS2QAP(*r1cs, F(1234567890), field=F)
A, B, C, Z = QAP

In [11]:
s, H = proof(f.witness(3), A, B, C, Z, field=F)

In [12]:
verify(s, A, B, C, Z, H)

True

In [13]:
A, B, C, Z, H

([FiniteFieldCyclicSecp256k1::3887743605418039269280372630259922367454495505554664031988337729448475509214,
  FiniteFieldCyclicSecp256k1::0,
  FiniteFieldCyclicSecp256k1::16714881326536888345592935925617868994972710272076919887981372210859069288668,
  FiniteFieldCyclicSecp256k1::0,
  FiniteFieldCyclicSecp256k1::60250785280976383371408897174303669926836233820841713082475649255891253643170,
  FiniteFieldCyclicSecp256k1::39034551790650655156831872651186322785116494731100995729588216737355103651526,
  FiniteFieldCyclicSecp256k1::78389071119561206711206874981537726566185824650048997683849490786436948241145,
  FiniteFieldCyclicSecp256k1::75186694398213649299083654014761027718978235879357036130815190426526726563525,
  FiniteFieldCyclicSecp256k1::0,
  FiniteFieldCyclicSecp256k1::0,
  FiniteFieldCyclicSecp256k1::110148626288675876860096936579925384093535232869153051858824660397565608413344,
  FiniteFieldCyclicSecp256k1::103386533268428100802213429812686359923294696605281437418969976910656578366

## An inner product proof

In Groth’s paper, he presents the core algorithm, which probably- not-coincidentally is also the core of Bulletproofs. The inner product proof here uses all the same elements as we’ve discussed above, although in a slightly more complicated structure.

**It starts by assuming that the Prover has two vectors $x$ and $y$, and obviously knows the inner product of those, which we’ll now call $z$.**

$$
C_z=tH+zG\\
C_x=rH+\mathbf{xG}\\
C_y=sH+\mathbf{yG}
$$

### 1. The commitment step

Analogous to the $R$ value of above two, The Prover $P$ will need to send commitments to two nonce vectors ($\mathbf{x, y}$).

These nonce vectors will be called $\mathbf{d}_x, \mathbf{d}y$.

Instead of sending $\mathbf{d}_x , \mathbf{d}_y$ , the Prover will instead send Pedersen commitments to them:

In [25]:
from klefki.curves.secp256k1 import (
    EllipticCurveGroupSecp256k1 as ECG,
    FiniteFieldSecp256k1 as F,
    FiniteFieldCyclicSecp256k1 as CF
)
from functools import reduce
import random
from hashlib import sha256
G = ECG.G
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
def random_cf() -> CF: return CF(random.randint(1, N) % F.P)

In [26]:
m = 10
x = [random_cf() for i in range(0, m)]
y = [random_cf() for i in range(0, m)]

def inner_product(x: [CF], y: [CF]):
    return reduce(lambda x, y: x + y, map(lambda x: x[0] * x[1], zip(x, y)))

def vgm(a: [CF]) -> [ECG]:
    return reduce(lambda x,y: x+y,
                  list(map(lambda a: a[0] @ a[1], zip(Gs, a))))

def map2curve(x: CF):
    return G @ x

secret = "sec"
Gs = list(map(map2curve,
              (map(lambda x: int(x, 16), 
                  (map(lambda x: sha256(x.encode()).hexdigest(), sha256(secret.encode()).hexdigest()))))))


z = inner_product(x, y)
H = G @ random_cf()

t, r, s = random_cf(), random_cf(), random_cf()

C_z = H@t + G@z
C_x = H@r + vgm(x)
C_y = H@s + vgm(y)

$$
A_d = r_dH + \mathbf{d}_x\mathbf{G}\\
B_d = s_dH + \mathbf{d}_y\mathbf{G}
$$

$r_d , s_d$ will be random values as usual for Pedersen commitments.

Just as the ﬁnal Schnorr response is $ex + k$, so here our ﬁnal response(s) are of the form $e\mathbf{x} + \mathbf{d}$, more speciﬁcally, one for each: 

$$
e\mathbf{x} + \mathbf{d}_x\\
e\mathbf{y} + \mathbf{d}_y\\
$$

In [27]:
dx = [random_cf() for i in range(0, m)]
dy = [random_cf() for i in range(0, m)]
r_d, s_d = random_cf(), random_cf()

A_d = H @ r_d + vgm(dx)
B_d = H @ s_d + vgm(dy)

However, that’s not enough; we’re trying to prove an inner product, too.

What we’ll have to do also in the commitment step is to send a commitment to the expected inner product of this $blinded$ form of our vectors. The blinded form has already been mentioned as $e\mathbf{x} + \mathbf{d}_x$ , $e\mathbf{y} + \mathbf{d}_y$ , but we don’t yet know the challenge $e$, so we have to factor that out somehow.

Now, **$e$ is a linear factor** in each of these terms, so dot-product-ing them$(e\mathbf{x} + \mathbf{d}_x)\cdot(e\mathbf{y} + \mathbf{d}_y)$will result in a **quadratic in $e$**, so there will be three coeﬃcients, and we’ll therefore need to provide commitments in advance for each of these three coeﬃcients.

\begin{align*}
(e\mathbf{x} + \mathbf{d}_x)·(e\mathbf{y} + \mathbf{d}_y) &= e\mathbf{x}·(e\mathbf{y}+\mathbf{d}_y) + \mathbf{d}_x·(e\mathbf{y}+\mathbf{d}_y) \\
&= e^2\mathbf{x} · \mathbf{y} + e^2(\mathbf{x} ·\mathbf{d}_y + \mathbf{y}· \mathbf{d}_x) + \mathbf{d}_x· \mathbf{d}_y
\end{align*}


$$
C_1 = t_1H + (\mathbf{x}·\mathbf{d}_y+\mathbf{y}·\mathbf{d}_x)G\\
C_0 = t_0H+(\mathbf{d}_x·\mathbf{d}_y)G\\
$$

Thus we got (Comm stand for Pedersen commitment) :
$$
Comm((e\mathbf{x} + \mathbf{d}_x)\cdot(e\mathbf{y} + \mathbf{d}_y)) = e^2C_z + e^2C_1 + C_0
$$

In [28]:
t1, t0 = random_cf(), random_cf()

C_1 = H@t1 + G@(inner_product(x, dy) + inner_product(y, dx))
C_0 = H@t0 + G@inner_product(dx, dy)


So to do all this in the commitment step, the Prover had to come up with 4 random scalars $r_d$ , $s_d$ , $t_1$ , $t_0$ and two random vectors $\mathbf{d}_x$ , $\mathbf{d}_y$ and then send 4 Pedersen commitments using this data: $A_d , B_d , C_1 , C_0$.

$$
(r_d , s_d , t_1 , t_0, A_d , B_d , C_1 , C_0)
$$


The commitment should be:

$$
(A_d, B_d, C_1, C_0)
$$

In [29]:
(A_d, B_d, C_1, C_0)

(EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::78100612623290373941044921488810286010640623338622508281181082724026593473057, FiniteFieldSecp256k1::91574966464075292231563687747103341520212393470471935652525339766664300861157),
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::34823029486988370380003976483881655108446969651844932218585069144705494562263, FiniteFieldSecp256k1::69507886371765581935971103710634349259962840939010852366268005685518481463721),
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::15550029510104143004807318233184509112961669378836849124403328836813702348633, FiniteFieldSecp256k1::52390706927545435330235144032895743450310552708515098501467998476538937071),
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::59739430043657505602341334361686613251212488478367658910140327528250589378362, FiniteFieldSecp256k1::113065198629723413967373997879787696519259848957691881473208293610955488476358))

### 2. The challange step

Nothing to discuss here – the Veriﬁer simply sends a single scalar value $e$.

In [31]:
e = random_cf()

### 3. The response step

The above detailed discussion will hopefully make the following set of data, sent by the Prover, less bewildering:

\begin{align*}
\mathbf{f}_x &= e\mathbf{x}+\mathbf{d}_x \\
\mathbf{f}_y &= e\mathbf{y}+\mathbf{d}_y \\
r_x&=er+r_d\\
s_y&=es+s_d\\
t_z&=e^2t+et_1+t_0
\end{align*}




In [32]:
fx = [e*x[i] + dx[i] for i in range(0, m)]
fy = [e*y[i] + dy[i] for i in range(0, m)]
rx = e*r + r_d
sy = e*s + s_d
tz = (e**2) * t + e * t1 + t0

note that here we are sending the blinded forms $f_x , f_y$ of the two vectors, not the Pedersen commitments – the idea is that the Veriﬁer will verify precisely by reconstructing the commitments and checking they match $C_x , C+y$ . Those two checks are:

\begin{align*}
eC_x+A_d &\stackrel{?}{=} r_xH+\mathbf{f}_x\mathbf{G}\\
eC_y+B_d &\stackrel{?}{=} s_yH+\mathbf{f}_y\mathbf{G}\\
t_zH+ (\mathbf{f}_x \cdot \mathbf{f}_y)G &\stackrel{?}{=}e^2C_z + eC_1 + C_0
\end{align*}



In [33]:
assert C_x @ e + A_d == H @ rx + vgm(fx)
assert C_y @ e + B_d == H @ sy + vgm(fy)

In [34]:
assert H @ tz + G @ inner_product(fx, fy) == C_z @ (e**2) + C_1 @ e + C_0